In [2]:
import os
import numpy as np
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [3]:
# Define the path to the train and test folders
train_path = 'C:/A/code/VSC/python/test/cropped_DevanagariHandwrittenCharacterDataset/Train/'
test_path = 'C:/A/code/VSC/python/test/cropped_DevanagariHandwrittenCharacterDataset/Test/'

# Define a function to load and preprocess the images
def load_images(path):
    # Create empty lists to store the images and labels
    images = []
    labels = []

    # Loop through each folder (which represents a class) in the path
    for folder in os.listdir(path):
        # Get the path to the folder
        folder_path = os.path.join(path, folder)
        # Loop through each image in the folder
        for file in os.listdir(folder_path):
            # Get the path to the image file
            img_path = os.path.join(folder_path, file)
            # Load the image using OpenCV
            img = cv2.imread(img_path)
            # Convert the image to grayscale
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # Normalize the pixel values of the images
            img = img.astype('float32') / 255.0
            # Append the image to the images list
            images.append(img)
            # Append the label to the labels list
            labels.append(folder)

    # Convert the images and labels lists to NumPy arrays
    images = np.array(images)
    labels = np.array(labels)
    
    # Return the images and labels arrays
    return images, labels

# Load the train images and labels
train_images, train_labels = load_images(train_path)

# Load the test images and labels
test_images, test_labels = load_images(test_path)

In [4]:
print(train_images.shape)

(78200, 28, 28)


In [5]:
print(train_labels.shape)

(78200,)


In [6]:
print(train_labels[0])

character_10_yna


In [7]:
print(test_labels[0])

character_10_yna


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [9]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Convert the string labels to integer labels
label_encoder = LabelEncoder()
train_labels_int = label_encoder.fit_transform(train_labels)

In [10]:
unique_values, counts = np.unique(train_labels_int, return_counts=True)
for value, count in zip(unique_values, counts):
    print(f"{value} occurs {count} times")

0 occurs 1700 times
1 occurs 1700 times
2 occurs 1700 times
3 occurs 1700 times
4 occurs 1700 times
5 occurs 1700 times
6 occurs 1700 times
7 occurs 1700 times
8 occurs 1700 times
9 occurs 1700 times
10 occurs 1700 times
11 occurs 1700 times
12 occurs 1700 times
13 occurs 1700 times
14 occurs 1700 times
15 occurs 1700 times
16 occurs 1700 times
17 occurs 1700 times
18 occurs 1700 times
19 occurs 1700 times
20 occurs 1700 times
21 occurs 1700 times
22 occurs 1700 times
23 occurs 1700 times
24 occurs 1700 times
25 occurs 1700 times
26 occurs 1700 times
27 occurs 1700 times
28 occurs 1700 times
29 occurs 1700 times
30 occurs 1700 times
31 occurs 1700 times
32 occurs 1700 times
33 occurs 1700 times
34 occurs 1700 times
35 occurs 1700 times
36 occurs 1700 times
37 occurs 1700 times
38 occurs 1700 times
39 occurs 1700 times
40 occurs 1700 times
41 occurs 1700 times
42 occurs 1700 times
43 occurs 1700 times
44 occurs 1700 times
45 occurs 1700 times


In [11]:
# Convert the integer labels to one-hot encoding
train_labels_onehot = to_categorical(train_labels_int, num_classes=46)

# Split the data into the five quadrants
x_train_tl = train_images[:,:14,:14].reshape(-1, 14, 14, 1)
x_train_tr = train_images[:,:14,14:].reshape(-1, 14, 14, 1)
x_train_bl = train_images[:,14:,:14].reshape(-1, 14, 14, 1)
x_train_br = train_images[:,14:,14:].reshape(-1, 14, 14, 1)
x_train_c = train_images[:,7:21,7:21].reshape(-1, 14, 14, 1)

In [12]:
print(train_labels_int.shape)
print(train_labels_onehot.shape)
print(train_labels_onehot[0])
type(train_labels_onehot)

(78200,)
(78200, 46)
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


numpy.ndarray

In [13]:
print(train_images.shape)
print(x_train_tl.shape)

(78200, 28, 28)
(78200, 14, 14, 1)


In [57]:
# CNN for top left quadrant

# Define the input shape for each quadrant
input_shape = (14, 14, 1)

# Create a sequential model for the top left quadrant
model_tl = Sequential()

# Add a convolutional layer with 16 filters, 3x3 kernel size, and sigmoid activation
model_tl.add(Conv2D(16, kernel_size=(3, 3), activation='sigmoid', input_shape=input_shape))

# Add a max pooling layer with 2x2 pool size
model_tl.add(MaxPooling2D(pool_size=(2, 2)))

# Add a convolutional layer with 32 filters, 3x3 kernel size, and sigmoid activation
model_tl.add(Conv2D(32, kernel_size=(3, 3), activation='sigmoid'))

# Add a max pooling layer with 2x2 pool size
model_tl.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the previous layer
model_tl.add(Flatten())

# Add a dense layer with 512 units and relu activation
model_tl.add(Dense(512, activation='relu'))

# Add the output layer with 46 units and softmax activation
model_tl.add(Dense(46, activation='softmax'))

# Compile the model with categorical crossentropy loss and Adam optimizer
model_tl.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model_tl.summary()

# Train the model on the top left quadrant dataset
model_tl.fit(x_train_tl, train_labels_onehot, batch_size=32, epochs=10, validation_split=0.1)


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 12, 12, 16)        160       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 6, 6, 16)         0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 4, 4, 32)          4640      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 2, 2, 32)         0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 512)              

In [58]:
# CNN for top right quadrant

# Define the input shape for each quadrant
input_shape = (14, 14, 1)

# Create a sequential model for the top left quadrant
model_tr = Sequential()

# Add a convolutional layer with 16 filters, 3x3 kernel size, and sigmoid activation
model_tr.add(Conv2D(16, kernel_size=(3, 3), activation='sigmoid', input_shape=input_shape))

# Add a max pooling layer with 2x2 pool size
model_tr.add(MaxPooling2D(pool_size=(2, 2)))

# Add a convolutional layer with 32 filters, 3x3 kernel size, and sigmoid activation
model_tr.add(Conv2D(32, kernel_size=(3, 3), activation='sigmoid'))

# Add a max pooling layer with 2x2 pool size
model_tr.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the previous layer
model_tr.add(Flatten())

# Add a dense layer with 512 units and relu activation
model_tr.add(Dense(512, activation='relu'))

# Add the output layer with 46 units and softmax activation
model_tr.add(Dense(46, activation='softmax'))

# Compile the model with categorical crossentropy loss and Adam optimizer
model_tr.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model_tr.summary()

# Train the model on the top left quadrant dataset
model_tr.fit(x_train_tr, train_labels_onehot, batch_size=32, epochs=10, validation_split=0.1)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 12, 12, 16)        160       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 6, 6, 16)         0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 4, 4, 32)          4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 2, 2, 32)         0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 128)               0         
                                                                 
 dense_12 (Dense)            (None, 512)              

In [59]:
# CNN for bottom left quadrant

# Define the input shape for each quadrant
input_shape = (14, 14, 1)

# Create a sequential model for the top left quadrant
model_bl = Sequential()

# Add a convolutional layer with 16 filters, 3x3 kernel size, and sigmoid activation
model_bl.add(Conv2D(16, kernel_size=(3, 3), activation='sigmoid', input_shape=input_shape))

# Add a max pooling layer with 2x2 pool size
model_bl.add(MaxPooling2D(pool_size=(2, 2)))

# Add a convolutional layer with 32 filters, 3x3 kernel size, and sigmoid activation
model_bl.add(Conv2D(32, kernel_size=(3, 3), activation='sigmoid'))

# Add a max pooling layer with 2x2 pool size
model_bl.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the previous layer
model_bl.add(Flatten())

# Add a dense layer with 512 units and relu activation
model_bl.add(Dense(512, activation='relu'))

# Add the output layer with 46 units and softmax activation
model_bl.add(Dense(46, activation='softmax'))

# Compile the model with categorical crossentropy loss and Adam optimizer
model_bl.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model_bl.summary()

# Train the model on the top left quadrant dataset
model_bl.fit(x_train_bl, train_labels_onehot, batch_size=32, epochs=10, validation_split=0.1)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 12, 12, 16)        160       
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 6, 6, 16)         0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 4, 4, 32)          4640      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 2, 2, 32)         0         
 g2D)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 128)               0         
                                                                 
 dense_14 (Dense)            (None, 512)              

In [60]:
# CNN for bottom right quadrant

# Define the input shape for each quadrant
input_shape = (14, 14, 1)

# Create a sequential model for the top left quadrant
model_br = Sequential()

# Add a convolutional layer with 16 filters, 3x3 kernel size, and sigmoid activation
model_br.add(Conv2D(16, kernel_size=(3, 3), activation='sigmoid', input_shape=input_shape))

# Add a max pooling layer with 2x2 pool size
model_br.add(MaxPooling2D(pool_size=(2, 2)))

# Add a convolutional layer with 32 filters, 3x3 kernel size, and sigmoid activation
model_br.add(Conv2D(32, kernel_size=(3, 3), activation='sigmoid'))

# Add a max pooling layer with 2x2 pool size
model_br.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the previous layer
model_br.add(Flatten())

# Add a dense layer with 512 units and relu activation
model_br.add(Dense(512, activation='relu'))

# Add the output layer with 46 units and softmax activation
model_br.add(Dense(46, activation='softmax'))

# Compile the model with categorical crossentropy loss and Adam optimizer
model_br.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model_br.summary()

# Train the model on the top left quadrant dataset
model_br.fit(x_train_br, train_labels_onehot, batch_size=32, epochs=10, validation_split=0.1)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 12, 12, 16)        160       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 6, 6, 16)         0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 4, 4, 32)          4640      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 2, 2, 32)         0         
 g2D)                                                            
                                                                 
 flatten_8 (Flatten)         (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 512)              

In [61]:
# CNN for center quadrant

# Define the input shape for each quadrant
input_shape = (14, 14, 1)

# Create a sequential model for the top left quadrant
model_c = Sequential()

# Add a convolutional layer with 16 filters, 3x3 kernel size, and sigmoid activation
model_c.add(Conv2D(16, kernel_size=(3, 3), activation='sigmoid', input_shape=input_shape))

# Add a max pooling layer with 2x2 pool size
model_c.add(MaxPooling2D(pool_size=(2, 2)))

# Add a convolutional layer with 32 filters, 3x3 kernel size, and sigmoid activation
model_c.add(Conv2D(32, kernel_size=(3, 3), activation='sigmoid'))

# Add a max pooling layer with 2x2 pool size
model_c.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the previous layer
model_c.add(Flatten())

# Add a dense layer with 512 units and relu activation
model_c.add(Dense(512, activation='relu'))

# Add the output layer with 46 units and softmax activation
model_c.add(Dense(46, activation='softmax'))

# Compile the model with categorical crossentropy loss and Adam optimizer
model_c.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model_c.summary()

# Train the model on the top left quadrant dataset
model_c.fit(x_train_c, train_labels_onehot, batch_size=32, epochs=10, validation_split=0.1)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 12, 12, 16)        160       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 6, 6, 16)         0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 4, 4, 32)          4640      
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 2, 2, 32)         0         
 g2D)                                                            
                                                                 
 flatten_9 (Flatten)         (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 512)              

In [66]:
from tensorflow.keras.models import Model

# create new models to extract features from the last dense layer
model_tl_features = Model(inputs=model_tl.input, outputs=model_tl.get_layer('dense_10').output)
model_tr_features = Model(inputs=model_tr.input, outputs=model_tr.get_layer('dense_12').output)
model_bl_features = Model(inputs=model_bl.input, outputs=model_bl.get_layer('dense_14').output)
model_br_features = Model(inputs=model_br.input, outputs=model_br.get_layer('dense_16').output)
model_c_features = Model(inputs=model_c.input, outputs=model_c.get_layer('dense_18').output)

# extract features from the last dense layer for each quadrant
tl_features = model_tl_features.predict(x_train_tl)
tr_features = model_tr_features.predict(x_train_tr)
bl_features = model_bl_features.predict(x_train_bl)
br_features = model_br_features.predict(x_train_br)
center_features = model_c_features.predict(x_train_c)

# concatenate the features from all 5 quadrants into a single input
features = np.concatenate((tl_features, tr_features, bl_features, br_features, center_features), axis=1)

2444/2444 [==============================] - 8s 3ms/step


In [1]:
print(model_tl_features.summary())

NameError: name 'model_tl_features' is not defined

In [70]:
print(x_train_tl.shape)
print(model_tl_features)
print(tl_features.shape)
print(features.shape)

(78200, 14, 14, 1)
(78200, 512)
(78200, 2560)


In [73]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# define the DNN classifier model
classifier = Sequential()
classifier.add(Dense(2560, activation='relu', input_shape=(features.shape[1],)))
classifier.add(Dropout(0.5))
classifier.add(Dense(1000, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(1000, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(1000, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(46, activation='softmax'))

# compile the model
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = classifier.fit(features, train_labels_onehot, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
1955/1955 [==============================] - 561s 287ms/step - loss: 0.9240 - accuracy: 0.7403 - val_loss: 23.2201 - val_accuracy: 0.0212
Epoch 2/10
1955/1955 [==============================] - 458s 234ms/step - loss: 0.5411 - accuracy: 0.8641 - val_loss: 25.0291 - val_accuracy: 0.0215
Epoch 3/10
1955/1955 [==============================] - 372s 190ms/step - loss: 0.5124 - accuracy: 0.8801 - val_loss: 28.4862 - val_accuracy: 0.0214
Epoch 4/10
1955/1955 [==============================] - 329s 168ms/step - loss: 0.5118 - accuracy: 0.8855 - val_loss: 38.6992 - val_accuracy: 0.0213
Epoch 5/10
1955/1955 [==============================] - 272s 139ms/step - loss: 0.5221 - accuracy: 0.8865 - val_loss: 47.5726 - val_accuracy: 0.0215
Epoch 6/10
1955/1955 [==============================] - 285s 146ms/step - loss: 0.5388 - accuracy: 0.8876 - val_loss: 65.1303 - val_accuracy: 0.0214
Epoch 7/10
1955/1955 [==============================] - 284s 145ms/step - loss: 0.5425 - accuracy: 0.8885 

In [ ]:
# Preprocessing the test data for prediction and testing


In [74]:
test_labels_int = label_encoder.fit_transform(test_labels)

In [75]:
# Convert the integer labels to one-hot encoding
test_labels_onehot = to_categorical(test_labels_int, num_classes=46)

# Split the data into the five quadrants
x_test_tl = test_images[:,:14,:14].reshape(-1, 14, 14, 1)
x_test_tr = test_images[:,:14,14:].reshape(-1, 14, 14, 1)
x_test_bl = test_images[:,14:,:14].reshape(-1, 14, 14, 1)
x_test_br = test_images[:,14:,14:].reshape(-1, 14, 14, 1)
x_test_c = test_images[:,7:21,7:21].reshape(-1, 14, 14, 1)

In [76]:
# extract features from the last dense layer for each quadrant
tl_features_test = model_tl_features.predict(x_test_tl)
tr_features_test = model_tr_features.predict(x_test_tr)
bl_features_test = model_bl_features.predict(x_test_bl)
br_features_test = model_br_features.predict(x_test_br)
center_features_test = model_c_features.predict(x_test_c)

# concatenate the features from all 5 quadrants into a single input
features_test = np.concatenate((tl_features_test, tr_features_test, bl_features_test, br_features_test, center_features_test), axis=1)

432/432 [==============================] - 1s 1ms/step


In [77]:
print(x_test_tl.shape)
print(model_tl_features)
print(tl_features_test.shape)
print(features_test.shape)

(13800, 14, 14, 1)
(13800, 512)
(13800, 2560)


In [78]:
# Save the model to disk
classifier.save("dnn.h5")


In [79]:
test_predictions = classifier.predict(features_test)

432/432 [==============================] - 7s 17ms/step


In [81]:
print(test_predictions.shape)

(13800, 46)


In [86]:
print(test_predictions[0])

[1.0000000e+00 0.0000000e+00 0.0000000e+00 7.1565388e-35 0.0000000e+00
 4.0553214e-34 7.5576329e-34 9.4512729e-33 0.0000000e+00 3.3397569e-38
 1.3861458e-29 6.3316524e-24 7.5105302e-26 7.2752837e-28 4.8854229e-34
 1.6285259e-14 1.9164847e-17 1.1292561e-30 0.0000000e+00 7.5905522e-24
 1.9051088e-37 3.2200462e-37 1.7306249e-27 1.6286748e-29 2.0155091e-20
 0.0000000e+00 3.8113278e-24 9.5887772e-15 1.9335126e-18 1.9375273e-20
 0.0000000e+00 3.4137172e-31 9.1156868e-27 0.0000000e+00 3.5423023e-10
 6.9995083e-28 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00]


In [84]:
# Evaluate the model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(test_labels_int, test_predictions.argmax(axis=1))
precision = precision_score(test_labels_int, test_predictions.argmax(axis=1), average='weighted', zero_division=1)
recall = recall_score(test_labels_int, test_predictions.argmax(axis=1), average='weighted')
f1 = f1_score(test_labels_int, test_predictions.argmax(axis=1), average='weighted')

In [88]:
print(test_labels_int[0])
print(test_predictions.argmax(axis=1)[0])

0
0


In [85]:
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Accuracy: 0.7683333333333333
Precision: 0.8549186034032531
Recall: 0.7683333333333333
F1 Score: 0.6999743222420854


In [95]:
# assume y_true and y_pred are the ground truth and predictions, respectively
precision = precision_score(test_labels_int, test_predictions.argmax(axis=1), average='weighted', zero_division=0, labels=np.unique(test_predictions))

# get the list of labels for which precision is being computed
labels = np.unique(test_predictions.argmax(axis=1))

# find the labels that were not predicted
missing_labels = np.setdiff1d(np.unique(test_labels_int), labels)

In [92]:
print(missing_labels)

[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45]


In [96]:
print(labels)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36]


In [97]:
print(labels.shape)

(37,)
